# 戦略・銘柄毎のシグナル判定（株価日足・ブログ用）

In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_ta as ta
import pickle
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

## シグナル判定で用いる関数の定義

In [2]:
# クロスオーバーとクロスアンダー関数
def crossover(series1, series2):
    # Ensure the inputs are numpy arrays for element-wise operations
    series1 = np.array(series1)
    series2 = np.array(series2)
    # Calculate the crossover: (previous series1 < previous series2) and (current series1 > current series2)
    cross = (series1[:-1] < series2[:-1]) & (series1[1:] > series2[1:])
    # Append a False at the start since there is no crossover detection at the first element
    return np.append(False, cross)

def overthreshold(series1, x):
    series1 = np.array(series1)
    series2 = np.full(series1.shape[0], x)
    # Calculate the crossover: (previous series1 < previous series2) and (current series1 > current series2)
    cross = (series1[:-1] < series2[:-1]) & (series1[1:] > series2[1:])
    # Append a False at the start since there is no crossover detection at the first element
    return np.append(False, cross)

def crossunder(series1, series2):
    # Ensure the inputs are numpy arrays for element-wise operations
    series1 = np.array(series1)
    series2 = np.array(series2)
    # Calculate the crossunder: (previous series1 > previous series2) and (current series1 < current series2)
    cross = (series1[:-1] > series2[:-1]) & (series1[1:] < series2[1:])
    # Append a False at the start since there is no crossunder detection at the first element
    return np.append(False, cross)

def underthreshold(series1, x):
    series1 = np.array(series1)
    series2 = np.full(series1.shape[0], x)
    # Calculate the crossunder: (previous series1 > previous series2) and (current series1 < current series2)
    cross = (series1[:-1] > series2[:-1]) & (series1[1:] < series2[1:])
    # Append a False at the start since there is no crossunder detection at the first element
    return np.append(False, cross)

## シグナル判定をする戦略の定義

In [3]:
# 戦略の生成
def generate_strategy(df, ma_fast=9, ma_slow=21,
                      macd_fast=12, macd_slow=26, macd_signal=9, efi_period=2):
    df1 = df.copy()
    # pandas_taライブラリによる各種指標の計算
    df1['MA_Fast'] = df.ta.sma(close='Close', length=ma_fast)
    df1['MA_Slow'] = df.ta.sma(close='Close', length=ma_slow)
    df1.ta.macd(close='Close', fast=macd_fast, slow=macd_slow, signal=macd_signal, append=True)
    df1['EFI'] = ta.efi(df['Close'], df['Volume'], length=efi_period)
    # シグナルの判定
    # (1) 長短移動平均の位置関係
    df1['MA_buySignal'] = crossover(df1['MA_Fast'], df1['MA_Slow'])
    df1['MA_sellSignal'] = crossunder(df1['MA_Fast'], df1['MA_Slow'])
    # (2) MACDとMACDsignalの位置関係
    macd_name = f'{macd_fast}_{macd_slow}_{macd_signal}'
    df1['MACD_buySignal'] = crossover(df1[f'MACD_{macd_name}'], df1[f'MACDs_{macd_name}'])
    df1['MACD_sellSignal'] = crossunder(df1[f'MACD_{macd_name}'], df1[f'MACDs_{macd_name}'])
    # (3) EFIとゼロの位置関係
    df1['EFI_buySignal'] = overthreshold(df1['EFI'], 0)
    df1['EFI_sellSignal'] = underthreshold(df1['EFI'], 0)
    # 関数の戻り
    return df1.loc[:, ['timestamp','ticker','MA_buySignal','MA_sellSignal','MACD_buySignal',\
                       'MACD_sellSignal','EFI_buySignal','EFI_sellSignal']]

In [4]:
# コード辞書と銘柄リスト
jpx = {'1306.T':"TOPIX連動", '1321.T':'日経平均連動', '1545.T':'NASDAQ連動','1546.T':'ダウ工業株30種連動',\
       '1557.T':'SP500連動', '1678.T':'Nifty50連動'}
list_ticker = jpx.keys()

## データの読み込みとシグナル判定

In [5]:
# 期間の指定
end_date = datetime.today()
start_date = end_date - timedelta(days=730)

In [6]:
# 各銘柄についてシグナル判定を行い結果を縦連結する
df_recent = pd.DataFrame()
for symbol in list_ticker:
    # yahooサイトからデータをダウンロード
    data = yf.download(symbol, start_date, end_date)
    if len(data) > 0:
        data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
        data['ticker'] = symbol
        df = data.reset_index().rename(columns={'Date':'timestamp'})
        # シグナル判定の実施
        df_strategy = generate_strategy(df)
        # シグナル判定結果を元データに横連結する
        df = df.merge(df_strategy, how="inner", on=['timestamp','ticker'])
        # 一銘柄のシグナル判定結果の最終行のみを元のデータに縦連結する
        df_recent = pd.concat([df_recent,df.tail(1)],axis=0)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


## 結果の整理と表示

In [7]:
# コードから銘柄名を返す関数
def ticker_dict(df):
    return jpx.get(df)
# 銘柄名の付与
df_signal = df_recent.loc[:,['timestamp','ticker','Close',\
                'MA_buySignal','MA_sellSignal','MACD_buySignal','MACD_sellSignal',\
                'EFI_buySignal','EFI_sellSignal']].reset_index()
df_signal['cname'] = df_signal['ticker'].map(ticker_dict)

In [8]:
# シグナルメッセージの付与
df_signal["signal_str"] = np.nan
df_signal["signal_total"] = np.nan
for irow in range(len(df_signal)):
    str_signal = ''
    total_signal = 0
    if df_signal.MA_buySignal[irow]:
        str_signal += 'MA買い,'
        total_signal += 100
    if df_signal.MACD_buySignal[irow]:
        str_signal += 'MACD買い,'
        total_signal += 100
    if df_signal.EFI_buySignal[irow]:
        str_signal += 'EFI買い,'
        total_signal += 100
    df_signal.loc[irow,"signal_str"] = str_signal
    df_signal.loc[irow,"signal_total"] = total_signal

In [9]:
# シグナル一覧の表示
pd.set_option('display.max_rows', None)
df_signal['s_total'] = df_signal['signal_total'].astype('int')
df_tmp = df_signal[['timestamp', 'ticker', 'cname', 'signal_str', 's_total']]
df_table = df_tmp[df_tmp['s_total'] > 0].sort_values(by='s_total',ascending=False)

In [10]:
# バックテスト結果の読み込み
df_backtest = pd.read_parquet('osc_backtest.parquet').reset_index()

In [11]:
# バックテストのパフォーマンスの最小値が高い順に表示する
join_data = pd.merge(df_table, df_backtest, on='ticker', how='left')
join_data['median_sharpe'] = join_data[['MA_buy','MACD_buy','EFI_buy']].median(axis=1)
join_data.sort_values(by='median_sharpe',ascending=False)

,timestamp,ticker,cname,signal_str,s_total,EFI_buy,MACD_buy,MA_buy,median_sharpe
2,2025-04-15,1557.T,SP500連動,"EFI買い,",100,1.505403,1.211504,1.027208,1.211504
1,2025-04-15,1546.T,ダウ工業株30種連動,"EFI買い,",100,1.358137,1.133177,0.930172,1.133177
0,2025-04-15,1545.T,NASDAQ連動,"MACD買い,",100,0.912084,0.996333,0.287659,0.912084
3,2025-04-15,1678.T,Nifty50連動,"EFI買い,",100,0.723914,0.670928,0.988381,0.723914
